In [1]:
from ultralytics import YOLO
from prettytable import PrettyTable
import torch
import torch.nn as nn

In [2]:
yolov3_path = "models/yolov3u.pt"
yolov3 = YOLO(yolov3_path)
yolov5_path = "models/yolov5nu.pt"
yolov5 = YOLO(yolov5_path)
yolov8_path = "models/yolov8n.pt"
yolov8 = YOLO(yolov8_path)

In [12]:
def get_module_output_shapes(model, dummy_input):
    output_shapes = {}
    hooks = []
    def hook_fn(module, input, output):
        if not list(module.named_children()):
            if isinstance(output, (tuple, list)):
                out_shape = list(output[0].shape) if isinstance(output[0], torch.Tensor) else "多输出"
            else:
                out_shape = list(output.shape) if isinstance(output, torch.Tensor) else "非张量输出"
            output_shapes[id(module)] = out_shape
    for name, module in model.named_modules():
        if name != "" and not list(module.named_children()):
            hooks.append(module.register_forward_hook(hook_fn))
    with torch.no_grad():
        model(dummy_input)
    for h in hooks:
        h.remove()
    return output_shapes
def compute_num_params(module):
    return sum(p.numel() for p in module.parameters())
def get_model_table(model, dummy_input=None):
    table = PrettyTable()
    table.field_names = ["Layer Index", "Layer Name", "Layer Type", "Input Shape", "Output Shape", "Number of Parameters", "Note"]
    dummy_input = torch.randn(1, 3, 448, 448) if dummy_input is None else dummy_input
    idx = 0
    model.eval()
    output_shapes = get_module_output_shapes(model.model, dummy_input)
    for orig_idx, (name, module) in enumerate(model.model.named_modules()):
        if name == "" or module is model.model:
            continue
        if list(module.named_children()) and orig_idx != 1:
            continue
        all_params = sum(p.numel() for p in module.parameters())
        # 2. 处理参数形状（卷积层补充stride/padding/kernel_size）
        shape_list = [list(p.size()) for p in module.parameters()]
        if len(shape_list) > 3:
            shape_list = ["..."]
        # 卷积层特殊处理
        notes = ""
        if isinstance(module, (nn.Conv2d, nn.Conv1d, nn.Conv3d)):
            conv_info = {
                "kernel_size": module.kernel_size,
                "stride": module.stride,
                "padding": module.padding
            }
            notes = f"k={conv_info['kernel_size']}, s={conv_info['stride']}, p={conv_info['padding']}"
        
        # 3. 获取输出形状（从提前记录的字典中取，避免实时计算报错）
        output_shape = output_shapes.get(id(module), "---")
        if isinstance(output_shape, list):
            output_shape = str(output_shape)
        
        # 索引自增（避免跳过模块导致索引断层）
        idx += 1
        
        # 4. 添加行
        table.add_row([
            idx,
            name,
            module._get_name(),
            shape_list,
            output_shape,
            all_params,
            notes
        ])
    return table

In [13]:
table = get_model_table(yolov8)
print(table)

+-------------+-----------------------+-------------+------------------------+-------------------+----------------------+------------------------------+
| Layer Index |       Layer Name      |  Layer Type |      Input Shape       |    Output Shape   | Number of Parameters |             Note             |
+-------------+-----------------------+-------------+------------------------+-------------------+----------------------+------------------------------+
|      1      |         model         |  Sequential |        ['...']         |        ---        |       3157200        |                              |
|      2      |      model.0.conv     |    Conv2d   |    [[16, 3, 3, 3]]     | [1, 16, 224, 224] |         432          | k=(3, 3), s=(2, 2), p=(1, 1) |
|      3      |       model.0.bn      | BatchNorm2d |      [[16], [16]]      | [1, 16, 224, 224] |          32          |                              |
|      4      |      model.0.act      |     SiLU    |           []           |  [1